In [26]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Markdown

In [27]:
ticker_symbol = "AAPL"
expiration_index = 0

# Create and load the ticker
ticker = yf.Ticker(ticker_symbol)
expirations = ticker.options
selected_expiration = expirations[expiration_index]

print(f"Using expiration date: {selected_expiration}")

# Get option chain
option_chain = ticker.option_chain(selected_expiration)
calls = option_chain.calls
puts = option_chain.puts

# Normalize IV
def normalize_iv(df):
    df = df.copy()
    df['impliedVolatility'] = df['impliedVolatility'].astype(float)
    df['impliedVolatility'] = df['impliedVolatility'].apply(
        lambda x: x / 100 if x > 2 else x
    )
    return df

calls = normalize_iv(calls)
puts = normalize_iv(puts)

# Filter stirkes to get rid of outliers
current_price = ticker.info['regularMarketPrice']

calls = calls[(calls['strike'] >= current_price * 0.8) &
              (calls['strike'] <= current_price * 1.2)]
puts = puts[(puts['strike'] >= current_price * 0.8) &
            (puts['strike'] <= current_price * 1.2)]

# Further filtering of the dataframe
def filter_options(df):
    df = df[df['volume'] > 0]
    df = df[df['openInterest'] > 0]
    df = df[df['[bid'] > 0]
    df['spread'] = df['ask'] - df['bid']
    df = df[df['spread'] / df['bid'] < 0.5]
    return df

filtered_calls = filter_options(calls)
filtered_puts = filter_options(puts)

# Results
display(Markdown('Filtered Calls'))
display(filtered_calls[['contractSymbol', 'strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']].head())

display(Markdown('Filtered Puts'))
display(filtered_puts[['contractSymbol', 'strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']].head())

print(f'Number of filtered calls: {len(filtered_calls)}')
print(f'Number of filtered puts: {len(filtered_puts)}')

Using expiration date: 2025-08-01


Filtered Calls

,contractSymbol,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility
10,AAPL250801C00170000,170.0,40.17,38.10,40.80,5.0,56,1.628908
11,AAPL250801C00175000,175.0,35.00,32.25,36.05,5.0,39,1.136723
12,AAPL250801C00180000,180.0,32.30,28.40,30.85,18.0,390,1.351566
13,AAPL250801C00182500,182.5,28.20,25.75,28.65,51.0,11,1.290043
14,AAPL250801C00185000,185.0,24.85,23.30,25.90,53.0,97,1.141606


Filtered Puts

,contractSymbol,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility
12,AAPL250801P00175000,175.0,0.05,0.03,0.04,326.0,9490,0.992188
13,AAPL250801P00180000,180.0,0.07,0.06,0.07,460.0,4186,0.917970
14,AAPL250801P00182500,182.5,0.09,0.07,0.09,432.0,495,0.871095
15,AAPL250801P00185000,185.0,0.11,0.10,0.12,702.0,9518,0.835939
16,AAPL250801P00187500,187.5,0.17,0.14,0.17,774.0,2175,0.802736


Number of filtered calls: 26
Number of filtered puts: 26
